<a href="https://colab.research.google.com/github/GiovanniBorrelli/ClinicalDataSanitization/blob/main/SpacyExperimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Questo notebook contiene sperimentazioni su Spacy con NER, che pur ottenendo risultati discreti, sono di molto inferiori rispetto a BERT

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/NLProject/mtsamples.csv", engine='python', on_bad_lines='skip')

print("Head of mtsamples.csv:")
display(df.head())

Head of mtsamples.csv:


,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [ ]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz

In [ ]:
import spacy
nlp = spacy.load("en_ner_bc5cdr_md")

# Test veloce
doc = nlp("The patient was prescribed ibuprofen for her chronic migraines.")
print([(ent.text, ent.label_) for ent in doc.ents])


[('ibuprofen', 'CHEMICAL'), ('migraines', 'DISEASE')]


In [ ]:
# Carica il modello NER di spaCy
nlp = spacy.load("en_core_web_sm")

In [ ]:
import time
from tqdm import tqdm

# Prepara la lista testi, riempiendo eventuali NaN con stringhe vuote
texts = df['transcription'].fillna("").tolist()

start_time = time.time()

# Usa nlp.pipe con tqdm per mostrare la progress bar
docs = list(tqdm(nlp.pipe(texts, batch_size=50), total=len(texts)))

# Estrai entità
df['entities'] = [[(ent.text, ent.label_) for ent in doc.ents] for doc in docs]

end_time = time.time()
elapsed = end_time - start_time

print(f"Tempo totale per l'elaborazione di {len(texts)} trascrizioni: {elapsed:.2f} secondi")


100%|██████████| 4999/4999 [06:45<00:00, 12.32it/s]


Tempo totale per l'elaborazione di 4999 trascrizioni: 406.08 secondi


In [ ]:
print(df[['transcription', 'entities']].head())

                                       transcription  \
0  SUBJECTIVE:,  This 23-year-old white female pr...   
1  PAST MEDICAL HISTORY:, He has difficulty climb...   
2  HISTORY OF PRESENT ILLNESS: , I have seen ABC ...   
3  2-D M-MODE: , ,1.  Left atrial enlargement wit...   
4  1.  The left ventricular cavity size and wall ...   

                                            entities  
0  complaint of allergies, allergies, Claritin, Z...  
1  snoring, pains, knee pain, pain, ankle pain, g...  
2  weight loss, weight loss, alcohol, weight loss...  
3        atrial enlargement, pulmonary regurgitation  
4  hyperdynamic left ventricular systolic functio...  


In [ ]:
# Sostituisci i punti e virgola con virgole per uniformare la separazione
df['entities'] = df['entities'].str.replace(';', ',')

# Prendi solo le colonne necessarie
result_df = df[['transcription', 'entities']]

# Salva in Excel
result_df.to_excel("trascriptions_entities_unite_scispacy.xlsx", index=False)

print("File Excel salvato come 'trascriptions_entities_unite_scispacy.xlsx'")


File Excel salvato come 'trascriptions_entities_unite_scispacy.xlsx'
